In [1]:
# compute acceleration error 方法 demo

import torch
import numpy as np

'''
- joints_gt 和 joints_pred 数组的大小为: N*21*2
    - N 表示所有测试集视频总共的帧数
    - 21 表示每帧中有 21 个关键点坐标
    - 2 表示关键点坐标 x 和 y

- 返回的加速度误差 accel_error 数组的大小为: N-2

- compute_accel_error 方法需要结合 gt 关键点和 pred 关键点计算

- source code: https://github.com/akanazawa/human_dynamics/blob/0887f37464/src/evaluation/eval_util.py
'''

joints_gt = np.array([[[1, 2], [3, 8], [5, 6], [7, 8]], [[3, 4], [5, 6], [7, 8], [9, 11]], [[5, 6], [7, 8], [9, 10], [11, 12]], [[5, 6], [7, 8], [9, 10], [11, 12]]])
joints_pred = np.array([[[1, 2], [3, 4], [5, 6], [7, 8]], [[3, 4], [5, 6], [7, 8], [9, 11]], [[5, 6], [7, 8], [9, 10], [11, 12]], [[5, 6], [7, 8], [9, 10], [11, 12]]])

def compute_error_accel(joints_gt, joints_pred, vis=None):
    """
    Computes acceleration error:
        1/(n-2) \sum_{i=1}^{n-1} X_{i-1} - 2X_i + X_{i+1}
    Note that for each frame that is not visible, three entries in the
    acceleration error should be zero'd out.
    Args:
        joints_gt (Nx14x3).
        joints_pred (Nx14x3).
        vis (N).
    Returns:
        error_accel (N-2).
    """
    # (N-2)x14x3
    accel_gt = joints_gt[:-2] - 2 * joints_gt[1:-1] + joints_gt[2:]
    accel_pred = joints_pred[:-2] - 2 * joints_pred[1:-1] + joints_pred[2:]

    normed = np.linalg.norm(accel_pred - accel_gt, axis=2)

    if vis is None:
        new_vis = np.ones(len(normed), dtype=bool)
    else:
        invis = np.logical_not(vis)
        invis1 = np.roll(invis, -1)
        invis2 = np.roll(invis, -2)
        new_invis = np.logical_or(invis, np.logical_or(invis1, invis2))[:-2]
        new_vis = np.logical_not(new_invis)

    return np.mean(normed[new_vis], axis=1)

accel_error = compute_error_accel(joints_gt, joints_pred)
print(accel_error)

[1. 0.]
